In [2]:
import math

import tensorflow as tf
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.datasets import mnist
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, Dropout, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

In [3]:
NUM_CLASSES = 10

In [4]:
(x_train, y_train), (x_test, y_test) = mnist.load_data(path='mnist.npz')

x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)

x_train = x_train / 255.0
x_test = x_test / 255.0

y_train = to_categorical(y_train, NUM_CLASSES)
y_test = to_categorical(y_test, NUM_CLASSES)

print("x_train.shape = {}, y_train.shape = {}".format(x_train.shape, y_train.shape))
print("x_test.shape = {}, y_test.shape = {}".format(x_test.shape, y_test.shape))

x_train.shape = (60000, 28, 28, 1), y_train.shape = (60000, 10)
x_test.shape = (10000, 28, 28, 1), y_test.shape = (10000, 10)


In [5]:
inputs = Input(shape=(28, 28, 1), name='input')

x = Conv2D(24, kernel_size=(6, 6), strides=1)(inputs)
x = BatchNormalization(scale=False, beta_initializer=Constant(0.01))(x)
x = Activation('relu')(x)
x = Dropout(rate=0.25)(x)

x = Conv2D(48, kernel_size=(5, 5), strides=2)(x)
x = BatchNormalization(scale=False, beta_initializer=Constant(0.01))(x)
x = Activation('relu')(x)
x = Dropout(rate=0.25)(x)

x = Conv2D(64, kernel_size=(4, 4), strides=2)(x)
x = BatchNormalization(scale=False, beta_initializer=Constant(0.01))(x)
x = Activation('relu')(x)
x = Dropout(rate=0.25)(x)

x = Flatten()(x)
x = Dense(200)(x)
x = BatchNormalization(scale=False, beta_initializer=Constant(0.01))(x)
x = Activation('relu')(x)
x = Dropout(rate=0.25)(x)

predications = Dense(NUM_CLASSES, activation='softmax', name='output')(x)

model = Model(inputs=inputs, outputs=predications)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [6]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 23, 23, 24)        888       
_________________________________________________________________
batch_normalization (BatchNo (None, 23, 23, 24)        72        
_________________________________________________________________
activation (Activation)      (None, 23, 23, 24)        0         
_________________________________________________________________
dropout (Dropout)            (None, 23, 23, 24)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 10, 10, 48)        28848     
_________________________________________________________________
batch_normalization_1 (Batch (None, 10, 10, 48)        144   

In [7]:
lr_decay = lambda epoch: 0.0001 + 0.02 * math.pow(1.0 / math.e, epoch / 3.0)
decay_callback = LearningRateScheduler(lr_decay, verbose=1)

history = model.fit(x_train, y_train, batch_size=128, epochs=20, verbose=1, 
                    validation_data=(x_test, y_test), callbacks=[decay_callback])

Train on 60000 samples, validate on 10000 samples

Epoch 00001: LearningRateScheduler reducing learning rate to 0.0201.
Epoch 1/20
60000/60000 [==============================] - 91s 2ms/sample - loss: 0.1598 - accuracy: 0.9510 - val_loss: 0.0551 - val_accuracy: 0.982565

Epoch 00002: LearningRateScheduler reducing learning rate to 0.014430626211475785.
Epoch 2/20
60000/60000 [==============================] - 94s 2ms/sample - loss: 0.0649 - accuracy: 0.9803 - val_loss: 0.0345 - val_accuracy: 0.9897

Epoch 00003: LearningRateScheduler reducing learning rate to 0.01036834238065184.
Epoch 3/20
60000/60000 [==============================] - 104s 2ms/sample - loss: 0.0431 - accuracy: 0.9868 - val_loss: 0.0289 - val_accuracy: 0.9912

Epoch 00004: LearningRateScheduler reducing learning rate to 0.007457588823428847.
Epoch 4/20
60000/60000 [==============================] - 106s 2ms/sample - loss: 0.0344 - accuracy: 0.9893 - val_loss: 0.0178 - val_accuracy: 0.9940

Epoch 00005: LearningRateSch

In [8]:
model.save('mnist.h5')

In [2]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_keras_model_file('mnist.h5')
tflite_model = converter.convert()
open('mnist.tflite', 'wb').write(tflite_model)

AttributeError: type object 'TFLiteConverterV2' has no attribute 'from_keras_model_file'

In [3]:
# 저장한 파일로부터 모델 변환 후 다시 저장
converter = tf.lite.TFLiteConverter.from_keras_model_file('mnist.h5')
tflite_model = converter.convert()
with open('mnist.tflite', 'wb') as f:
    f.write(tflite_model)

#save_model('./model/mnist.h5', './model/mnist.tflite')

AttributeError: type object 'TFLiteConverterV2' has no attribute 'from_keras_model_file'

In [ ]:
try:
    from google.colab import files
    files.download('mnist.tflite')
except:
    print("Skip downloading")

In [5]:
# WHOLE MODEL
tflite_model = tf.keras.models.load_model('mnist.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(tflite_model)
tflite_save = converter.convert()
open("mnist.tflite", "wb").write(tflite_save)

INFO:tensorflow:Assets written to: C:\Users\Public\Documents\ESTsoft\CreatorTemp\tmps9e8j9bl\assets


1147380